In [1]:
import os
import json
from tqdm import tqdm
from dotenv import load_dotenv

from openai import OpenAI
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer

/home/codespace/.local/share/virtualenvs/running-assistant-rag-j-iIWklV/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [25]:
# load the running qa documents
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [26]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [27]:
documents[0]

{'text': 'Intervalltraining ist eine Trainingsmethode, bei der sich intensive Belastungsphasen mit Erholungsphasen abwechseln. Durch diese Wechsel zwischen hoher und niedriger Belastung werden Ausdauer und Schnelligkeit effektiv verbessert, da der Körper sowohl anaerob als auch aerob gefordert wird.',
 'section': 'Training',
 'question': 'Was ist Intervalltraining?',
 'course': 'running-assistant-rag'}

In [8]:
es_client = Elasticsearch('http://localhost:9200')

In [9]:
es_client.info()

ObjectApiResponse({'name': 'a8cb232ba2b1', 'cluster_name': 'docker-cluster', 'cluster_uuid': '-zlyx-UFQ6SmB_FcuOnFtw', 'version': {'number': '9.0.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '73f7594ea00db50aa7e941e151a5b3985f01e364', 'build_date': '2025-04-30T10:07:41.393025990Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [10]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "running-questions"

In [2]:
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings, request_timeout=60)

NameError: name 'es_client' is not defined

In [12]:
for doc in tqdm(documents):
    try:
        es_client.index(index=index_name, document=doc, request_timeout=60)
    except Exception as e:
        print(e)

  0%|          | 0/98 [00:00<?, ?it/s]/tmp/ipykernel_4735/1667570353.py:3: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es_client.index(index=index_name, document=doc, request_timeout=60)
100%|██████████| 98/98 [00:02<00:00, 34.44it/s]


In [13]:
query = 'Was ist Intervalltraining?'

In [14]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "running-assistant-rag"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [15]:
# search in running qa documents with elasticsearch
elastic_search('Wie fange ich mit dem Laufen an?')

[{'text': 'Barfußlaufen fördert die Fußmuskulatur, sollte aber langsam aufgebaut werden, um Verletzungen zu vermeiden.',
  'section': 'Training',
  'question': 'Wie beginnt man mit Barfußlaufen sicher?',
  'course': 'running-assistant-rag'},
 {'text': 'Das Tragen von reflektierender Kleidung erhöht die Sichtbarkeit und Sicherheit beim Laufen in der Dunkelheit.',
  'section': 'Ausrüstung',
  'question': 'Wie kann ich beim Laufen im Dunkeln sicherer sein?',
  'course': 'running-assistant-rag'},
 {'text': 'Der optimale Laufschuh sollte gut passen, eine ausreichende Dämpfung bieten und zu deinem Laufstil passen, um Verletzungen vorzubeugen.',
  'section': 'Ausrüstung',
  'question': 'Wie finde ich den richtigen Laufschuh?',
  'course': 'running-assistant-rag'},
 {'text': 'Das richtige Tempo finden ist wichtig, um das Rennen effektiv durchzuhalten und das Risiko eines Leistungseinbruchs zu minimieren.',
  'section': 'Wettkämpfe',
  'question': 'Wie finde ich das richtige Renntempo?',
  'cou

In [20]:
def build_prompt(query, search_results):
    prompt_template = """
You are a personal trainer specialized in running.

Answer the QUESTION at the end based on the full CONTEXT below, which contains multiple relevant FAQ entries.
Use **all available context entries** to form your answer. If multiple answers apply, summarize or combine them clearly.
Don't mention the Entry number like [Entry 5] in your answer.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context_entries = []
    
    for i, doc in enumerate(search_results, start=1):
        entry = (
            f"[Entry {i}]\n"
            f"Section: {doc['section']}\n"
            f"Question: {doc['question']}\n"
            f"Answer: {doc['text']}"
        )
        context_entries.append(entry)

    context = "\n\n".join(context_entries)
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [21]:
def llm(prompt):
    # uncomment to see full prompt
    # print(prompt)
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}],
        temperature=1.5,
        top_p=0.9,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        max_tokens=1000,
    )
    
    return response.choices[0].message.content

In [18]:
def rag_pipeline(query):
    context_docs = elastic_search(query)
    prompt = build_prompt(query, context_docs)
    answer = llm(prompt)
    return answer

In [19]:
query = 'Wie fange ich mit dem Laufen an?'

In [20]:
# answer using qa documents for rag
rag_pipeline(query)

'Um mit dem Laufen anzufangen, solltest du es langsam und schrittweise angehen, um Verletzungen zu vermeiden. Wähle den richtigen Laufschuh, der gut passt, eine ausreichende Dämpfung bietet und zu deinem Laufstil passt. Achte auch darauf, nach dem Laufen zu dehnen, um die Beweglichkeit zu fördern und die Regeneration zu unterstützen.'

In [23]:
# answer from gpt-4o without using rag
llm(query)

'Mit dem Laufen anzufangen ist eine großartige Entscheidung für Ihre Gesundheit und Ihr Wohlbefinden. Hier sind einige Tipps, um Ihnen den Einstieg zu erleichtern:\n\n1. **Ziele setzen:** Definieren Sie klare, realistische Ziele. Möchten Sie fitter werden, abnehmen oder an einem Wettlauf teilnehmen? Dies hilft Ihnen, motiviert zu bleiben.\n\n2. **Richtige Ausrüstung:** Investieren Sie in ein Paar gute Laufschuhe, die zu Ihrem Fußtyp und Laufstil passen. Fachgeschäfte bieten oft eine Beratung und Analyse an.\n\n3. **Langsam anfangen:** Beginnen Sie mit einem Wechsel aus Laufen und Gehen. Zum Beispiel: 1 Minute laufen, 2 Minuten gehen, und das für 20-30 Minuten. Steigern Sie langsam die Laufanteile.\n\n4. **Aufwärmen und Abkühlen:** Starten Sie jede Laufeinheit mit einem Aufwärmen, zum Beispiel leichtes Gehen und dynamische Dehnübungen. Vergessen Sie nicht, am Ende zu dehnen, um die Regeneration zu fördern.\n\n5. **Regelmäßigkeit:** Versuchen Sie, regelmäßig zu laufen, am besten 3-mal pr

# Implement Vector Search

In [21]:
model = SentenceTransformer("all-mpnet-base-v2")

In [22]:
#creating embeddings/dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

In [25]:
operations[0]

{'text': 'Intervalltraining ist eine Trainingsmethode, bei der sich intensive Belastungsphasen mit Erholungsphasen abwechseln. Durch diese Wechsel zwischen hoher und niedriger Belastung werden Ausdauer und Schnelligkeit effektiv verbessert, da der Körper sowohl anaerob als auch aerob gefordert wird.',
 'section': 'Training',
 'question': 'Was ist Intervalltraining?',
 'course': 'running-assistant-rag',
 'text_vector': [-0.04246851056814194,
  -0.06532343477010727,
  -0.0055569554679095745,
  -0.0009684512042440474,
  -0.03175415098667145,
  -0.007610880769789219,
  0.008620666339993477,
  0.038966089487075806,
  -0.01865781657397747,
  0.012907765805721283,
  0.02927362732589245,
  0.0635496973991394,
  -0.03963595628738403,
  0.009810736402869225,
  -0.021428510546684265,
  -0.08541204780340195,
  -0.005765752401202917,
  0.0073867253959178925,
  -0.0655638724565506,
  0.016591601073741913,
  0.03275738283991814,
  -0.03081679344177246,
  -0.023442065343260765,
  -0.004687634296715259

In [26]:
# create mappings and index
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

index_name = "running-questions"

In [27]:
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings, request_timeout=60)

/tmp/ipykernel_4735/1308020260.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es_client.indices.create(index=index_name, body=index_settings, request_timeout=60)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'running-questions'})

In [28]:
# add documents to index
for doc in tqdm(documents):
    try:
        es_client.index(index=index_name, document=doc, request_timeout=60)
    except Exception as e:
        print(e)

  0%|          | 0/98 [00:00<?, ?it/s]/tmp/ipykernel_4735/2787033556.py:4: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es_client.index(index=index_name, document=doc, request_timeout=60)
100%|██████████| 98/98 [00:03<00:00, 29.01it/s]


In [29]:
query = 'Was ist Intervalltraining?'
vector_search_term = model.encode(query)

In [30]:
search_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 3,
    "num_candidates": 10000, 
}

In [31]:
res = es_client.search(index=index_name, knn=search_query, source=["text", "section", "question", "course"])
res["hits"]["hits"]

[{'_index': 'running-questions',
  '_id': 'GlP_nJcB5tLelV9K5xL2',
  '_score': 0.8421757,
  '_source': {'text': 'Intervalltraining steigert die anaerobe Kapazität und verbessert die Schnelligkeit.',
   'section': 'Training',
   'question': 'Was bringt Intervalltraining beim Laufen?',
   'course': 'running-assistant-rag'}},
 {'_index': 'running-questions',
  '_id': 'w1P_nJcB5tLelV9K3BES',
  '_score': 0.8411081,
  '_source': {'text': 'Intervalltraining ist eine Trainingsmethode, bei der sich intensive Belastungsphasen mit Erholungsphasen abwechseln. Durch diese Wechsel zwischen hoher und niedriger Belastung werden Ausdauer und Schnelligkeit effektiv verbessert, da der Körper sowohl anaerob als auch aerob gefordert wird.',
   'section': 'Training',
   'question': 'Was ist Intervalltraining?',
   'course': 'running-assistant-rag'}},
 {'_index': 'running-questions',
  '_id': 'H1P_nJcB5tLelV9K6BJ8',
  '_score': 0.8202045,
  '_source': {'text': 'Das Laufen im Intervall kann den Stoffwechsel an

In [32]:
def vector_search(query):
    vector_search_term = model.encode(query)
    search_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
    }

    response = es_client.search(index=index_name, knn=search_query, source=["text", "section", "question", "course"])
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [33]:
vector_search('Wie fange ich mit dem Laufen an?')

[{'text': 'Laufschuhe mit zu wenig Dämpfung können zu Gelenkschmerzen führen, besonders bei längeren Läufen.',
  'section': 'Ausrüstung',
  'question': 'Welche Folgen kann zu wenig Dämpfung bei Laufschuhen haben?',
  'course': 'running-assistant-rag'},
 {'text': 'Laufen verbessert die Herz-Kreislauf-Gesundheit durch Stärkung des Herzens und Senkung des Blutdrucks.',
  'section': 'Gesundheit',
  'question': 'Wie verbessert Laufen die Herzgesundheit?',
  'course': 'running-assistant-rag'},
 {'text': 'Laufschuhe sollten vor dem Kauf idealerweise abends anprobiert werden, da die Füße im Tagesverlauf anschwellen.',
  'section': 'Ausrüstung',
  'question': 'Wann ist die beste Tageszeit, um Laufschuhe anzuprobieren?',
  'course': 'running-assistant-rag'},
 {'text': 'Das Laufen auf der Mittelfußsohle wird oft als gesünder und effizienter angesehen als das Fersenlaufen.',
  'section': 'Technik',
  'question': 'Welche Lauftechnik ist gesünder: Mittelfuß- oder Fersenlauf?',
  'course': 'running-a

In [34]:
def rag_pipeline(query):
    print('using vector search')
    context_docs = vector_search(query)
    prompt = build_prompt(query, context_docs)
    answer = llm(prompt)
    return answer

In [35]:
query = 'Wie fange ich mit dem Laufen an?'

In [36]:
# answer using qa documents for rag
rag_pipeline(query)

using vector search


'Um mit dem Laufen zu beginnen, ist es wichtig, einige grundlegende Überlegungen anzustellen. Zunächst solltest du in gute Laufschuhe investieren, die genügend Dämpfung bieten, um Gelenkschmerzen zu vermeiden, besonders wenn du längere Strecken planst. Es wird empfohlen, die Laufschuhe abends anzuprobieren, da die Füße im Laufe des Tages anschwellen, und sie vor dem regulären Training einzulaufen, um Blasen zu vermeiden. Bei der Technik kann es hilfreich sein, sich auf das Laufen auf der Mittelfußsohle zu konzentrieren, da dies oft als gesünder und effizienter gilt als das Fersenlaufen. Wenn du diese Tipps befolgst, legst du einen soliden Grundstein für deinen Start ins Laufen.'

# Rag with qdrant

In [7]:
from qdrant_client import QdrantClient, models

In [8]:
qd_client = QdrantClient("http://localhost:6333")

In [9]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [10]:
collection_name = "running-faq"

In [38]:
qd_client.delete_collection(collection_name=collection_name)

True

In [39]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [ ]:
# qd_client.create_payload_index(
#     collection_name=collection_name,
#     field_name="course",
#     field_schema="keyword"
# )

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [40]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [41]:
points[0]

PointStruct(id=0, vector=Document(text='Was ist Intervalltraining? Intervalltraining ist eine Trainingsmethode, bei der sich intensive Belastungsphasen mit Erholungsphasen abwechseln. Durch diese Wechsel zwischen hoher und niedriger Belastung werden Ausdauer und Schnelligkeit effektiv verbessert, da der Körper sowohl anaerob als auch aerob gefordert wird.', model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'text': 'Intervalltraining ist eine Trainingsmethode, bei der sich intensive Belastungsphasen mit Erholungsphasen abwechseln. Durch diese Wechsel zwischen hoher und niedriger Belastung werden Ausdauer und Schnelligkeit effektiv verbessert, da der Körper sowohl anaerob als auch aerob gefordert wird.', 'section': 'Training', 'question': 'Was ist Intervalltraining?', 'course': 'running-assistant-rag'})

In [42]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [43]:
question = 'Wie fange ich mit dem Laufen an?'

In [44]:
def vector_search(question):
    print('vector_search is used')
    
    # course = 'data-engineering-zoomcamp'
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        limit=5,
        with_payload=True
    )
    
    results = []
    
    for point in query_points.points:
        results.append(point.payload)
    
    return results

In [45]:
def rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [46]:
rag('Wie fange ich mit dem Laufen an?')

vector_search is used


'Um mit dem Laufen zu beginnen, ist es wichtig, zunächst auf die richtige Ausrüstung zu achten. Wählen Sie geeignete Laufschuhe basierend auf Ihrem Fußtyp und Laufstil, z.\u202fB. Stabilitätsschuhe, wenn Sie zu Überpronation neigen, oder neutrale Schuhe, wenn Sie einen normalen Fuß haben. Achten Sie darauf, Schuhe abends anzuprobieren, da Ihre Füße im Tagesverlauf anschwellen. Außerdem sollten die Schuhe genügend Dämpfung bieten, um Gelenkschmerzen zu vermeiden, besonders bei längeren Läufen.'

In [49]:
vector_search('Wie fange ich mit dem Laufen an?')

vector_search is used


[{'text': 'Das Tragen von reflektierender Kleidung erhöht die Sichtbarkeit und Sicherheit beim Laufen in der Dunkelheit.',
  'section': 'Ausrüstung',
  'question': 'Wie kann ich beim Laufen im Dunkeln sicherer sein?',
  'course': 'running-assistant-rag'},
 {'text': 'Laufen kann die Knochendichte erhöhen und somit Osteoporose vorbeugen.',
  'section': 'Gesundheit',
  'question': 'Wie wirkt sich Laufen auf die Knochengesundheit aus?',
  'course': 'running-assistant-rag'},
 {'text': 'Laufschuhe mit zu wenig Dämpfung können zu Gelenkschmerzen führen, besonders bei längeren Läufen.',
  'section': 'Ausrüstung',
  'question': 'Welche Folgen kann zu wenig Dämpfung bei Laufschuhen haben?',
  'course': 'running-assistant-rag'},
 {'text': 'Laufschuhe sollten je nach Fußtyp und Laufstil ausgewählt werden, z.\u202fB. Stabilitätsschuhe für Überpronierer oder neutrale Schuhe für Normalfußläufer.',
  'section': 'Ausrüstung',
  'question': 'Welche Laufschuhe eignen sich für Überpronierer?',
  'course':